In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from load_data import FD001_dataset
from visualize_modified import visualize
from RUL_model import RUL_Model, RUL_Model_LearnableStates
from torch.utils.tensorboard import SummaryWriter
from typing import Optional, Tuple, List
from tqdm.auto import tqdm

# Check if MPS is available and built
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Using MPS (Apple Silicon GPU)")
elif torch.cuda.is_available():
    # Fallback for systems with NVIDIA GPUs (though less relevant on a Mac)
    device = torch.device("cuda")
    print("Using CUDA")
else:
    device = torch.device("cpu")
    print("MPS or CUDA not available, using CPU")

DATA_DIR_PATH = '../CMAPSSData/'
BATCH_SIZE = 20

import plotly.express as px
import plotly.io as pio
# pio.templates.default = "none"
pio.templates.default = "plotly_dark"

N_EPOCH = 200  # NUM OF EPOCHS
RUL_UPPER_BOUND = 130  # UPPER BOUND OF RUL
LR = 0.001  # LEARNING RATE

Using MPS (Apple Silicon GPU)


In [3]:
train_dataset = FD001_dataset(data_type='train', rul_ub=RUL_UPPER_BOUND, norm_type='zscore')
test_dataset = FD001_dataset(data_type='test', rul_ub=RUL_UPPER_BOUND, norm_type='zscore')

In [4]:
def train_model(
    model: nn.Module,
    train_dataLoader: torch.utils.data.DataLoader,
    test_dataLoader: torch.utils.data.DataLoader,
    loss_fn: nn.Module, # IMPORTANT: Expects loss_fn with reduction='none'
    optimizer: torch.optim.Optimizer,
    device: torch.device, # Pass the device object
    writer: Optional[SummaryWriter] = None,
    num_epochs: int = 100,
    y_test: Optional[np.ndarray] = None, # Assuming y_test is for visualization comparison
    start_epoch: int = 1,
    patience: int = 10, # Number of epochs to wait for improvement before stopping
    best_model_path: str = 'best_model.pth' # Path to save the best model
    ) -> Tuple[float, float, int]:
    """
    Trains and validates a PyTorch model for sequence prediction (e.g., RUL).

    Args:
        model: The PyTorch model to train (should already be on 'device').
        train_dataLoader: DataLoader for training data. Yields (padded_sequences, lengths, padded_targets).
        test_dataLoader: DataLoader for validation data. Yields (sequences, targets).
                         Assumes fixed length or model handles variable length internally for validation.
        loss_fn: Loss function instance (MUST have reduction='none').
        optimizer: Optimizer instance.
        device: The torch.device (cuda, mps, cpu) to run training on.
        writer: Optional TensorBoard SummaryWriter instance.
        num_epochs: Total number of epochs to train for.
        y_test: Optional ground truth for visualization during validation.
        start_epoch: The starting epoch number (e.g., for resuming training).
        patience: Number of epochs with no validation loss improvement to trigger early stopping.
        best_model_path: File path to save the best performing model state_dict.

    Returns:
        Tuple containing:
            - Final training loss (average per sample).
            - Best validation loss achieved (RMSE on last time step).
            - The epoch number when training finished.
    """
    print(f"Using device: {device}")
    # --- Ensure loss function has reduction='none' ---
    try:
        if hasattr(loss_fn, 'reduction') and loss_fn.reduction != 'none':
            print(f"Warning: loss_fn.reduction is '{loss_fn.reduction}'. "
                  "For masked loss, it should be 'none'. Attempting to proceed, "
                  "but results might be incorrect if loss isn't element-wise.")
    except Exception as e:
        print(f"Could not check loss_fn.reduction: {e}. Assuming it's 'none'.")


    best_val_loss = float('inf')
    epochs_no_improve = 0
    current_batch_size = train_dataLoader.batch_size # Get batch size for printing

    for epoch in range(start_epoch, start_epoch + num_epochs):
        model.train()
        total_train_loss_sum = 0  # Sum of losses over valid elements in epoch
        total_valid_elements = 0  # Total number of valid (non-padded) elements

        # Optional: Use tqdm for a cleaner progress bar
        # train_iterator = tqdm(train_dataLoader, desc=f"Epoch {epoch: 3}/{num_epochs + start_epoch - 1} Training", leave=True)
        # for i, (x_padSeq, original_seq_lens, y_padSeq) in enumerate(train_iterator):

        print(f"Epoch {epoch: 3}/{num_epochs + start_epoch - 1} | Training...   ", end="")
        for i, (x_padSeq, original_seq_lens, y_padSeq) in enumerate(train_dataLoader):
            # Simple progress indicator
            print(f"\b\b{(i+1)*BATCH_SIZE:02}", end='')

            # --- Move data to device ---
            # NOTE: original_seq_lens stays on CPU for masking/packing
            x_padSeq = x_padSeq.to(device)
            y_padSeq = y_padSeq.to(device)
            # original_seq_lens = original_seq_lens.to(device) # Keep on CPU

            optimizer.zero_grad()
            # --- Forward pass ---
            # Assumes model handles lengths correctly if passed
            y_pred = model(x_padSeq, original_seq_lens.cpu()) # Ensure lengths are CPU for model

            # --- Masked Loss Calculation ---
            loss_elementwise = loss_fn(y_pred, y_padSeq) # Needs reduction='none'
            # 1. Create the mask (True/1 for non-padded, False/0 for padded) on CPU
            max_len = y_pred.size(1) # Get max sequence length from output
            indices_cpu = torch.arange(max_len) # Shape: [max_len] on CPU
            # Expand indices to batch shape and compare with lengths (on CPU)
            mask_cpu = indices_cpu.expand(x_padSeq.size(0), -1) < original_seq_lens.unsqueeze(1) # Shape: [batch, max_len] (boolean) on CPU
            mask = mask_cpu.unsqueeze(-1).to(device) # Shape: [batch, max_len, 1] (boolean) on Device

            # 2. Apply mask and calculate sum of loss over valid elements
            # Ensure mask dtype is compatible with multiplication (convert boolean to float/int)
            masked_loss = loss_elementwise * mask.float() # Or mask.int()
            batch_total_loss = torch.sum(masked_loss)
            num_valid_in_batch = torch.sum(mask)

            # Avoid division by zero if a batch has no valid elements (e.g., all padding)
            if num_valid_in_batch > 0:
                 # Optional: Calculate mean loss for the batch for debugging/logging
                #  batch_mean_loss = batch_total_loss / num_valid_in_batch

                 # --- Backward pass and optimize ---
                 batch_total_loss.backward()
                 # Optional: Gradient clipping
                 # torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                 optimizer.step()

                 total_train_loss_sum += batch_total_loss.item()
                 total_valid_elements += num_valid_in_batch.item()
            else:
                 # Handle case where batch has no valid elements (skip backward/step)
                 print(f"Warning: Batch {i+1} has no valid elements based on lengths.")
                 pass # No loss contribution, no backward pass

        # --- End of Training Epoch ---
        # Calculate average loss per valid element for the epoch
        avg_train_loss = total_train_loss_sum / total_valid_elements if total_valid_elements > 0 else 0
        print(f" | Train Loss (avg/element): {avg_train_loss:.4f} | ", end="")

        if writer is not None:
            writer.add_scalar("Loss/train_avg_per_element", avg_train_loss, epoch)

        #### VALIDATION ####
        model.eval()
        total_val_sq_error_sum = 0.0
        total_val_samples = 0 # Count samples based on test_dataLoader size
        val_predictions_list = [] # Store predictions (on CPU)

        with torch.no_grad():
            # Use validation set for evaluating performance
            for i, (x_padSeq, original_seq_lens, y_padSeq) in enumerate(test_dataLoader):
                # Move validation data to device
                x_padSeq = x_padSeq.to(device)
                y_padSeq = y_padSeq.to(device)

                # Validation sequences can be variable, pass lengths from test_dataLoader
                # y_pred_val = model(x_val, val_lengths.cpu())
                y_pred_val = model(x_padSeq, original_seq_lens.cpu()) # Assuming model handles lengths=None if needed
                
                # We need the output of the *last time step* of the *last* LSTM layer
                batch_size = x_padSeq.size(0)
                # Gather the outputs at the actual sequence lengths
                last_seq_idxs = (original_seq_lens - 1).to(device) # Move indices to correct device
                # Create batch indices [0, 1, ..., batch_size-1]
                batch_indices = torch.arange(0, batch_size).to(device)
                # Index the target and prediction using batch_indices and last_seq_idxs
                y_val_last_step = y_padSeq[batch_indices, last_seq_idxs, :]
                pred_last_step = y_pred_val[batch_indices, last_seq_idxs, :]
                
                sq_error = torch.pow(pred_last_step - y_val_last_step, 2)
                total_val_sq_error_sum += torch.sum(sq_error).item() # Sum squared errors in batch
                total_val_samples += batch_size # Add number of samples in this batch

                # Store predictions (move to CPU) for potential visualization
                val_predictions_list.append(pred_last_step.cpu().numpy())

        # Calculate validation RMSE after iterating through all batches
        val_rmse = np.sqrt(total_val_sq_error_sum / total_val_samples) if total_val_samples > 0 else 0
        print(f"Val RMSE (last step): {val_rmse:.4f}")

        if writer is not None:
            writer.add_scalar("Loss/validation_RMSE_last_step", val_rmse, epoch)
            writer.flush()

        # --- Early Stopping & Model Saving ---
        if val_rmse < best_val_loss:
            print(f"Validation RMSE improved ({best_val_loss:.4f} --> {val_rmse:.4f}). Saving model...")
            best_val_loss = val_rmse
            torch.save(model.state_dict(), best_model_path)
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
            print(f"Validation RMSE did not improve for {epochs_no_improve} epoch(s). Best: {best_val_loss:.4f}")

        if epochs_no_improve >= patience:
            print(f"\n--- Early stopping triggered after {epoch} epochs ---")
            break

        # --- Visualization ---
        if epoch % 10 == 0 and y_test is not None and len(val_predictions_list) > 0:
             # Consolidate predictions from batches
             all_val_preds = np.concatenate(val_predictions_list, axis=0)
             # Call your visualization function
             visualize(all_val_preds, y_test, 100, val_rmse, epoch, writer)

    # --- End of Training ---
    if writer is not None:
        writer.flush()
        # writer.close() # Close writer if finished

    print(f"\nTraining finished after epoch {epoch}. Best Validation RMSE: {best_val_loss:.4f}")
    # Load the best model weights back before returning (optional)
    # model.load_state_dict(torch.load(best_model_path, map_location=device))
    return avg_train_loss, best_val_loss, epoch # Return last train loss, best val loss, final epoch

In [5]:
# *** Define a padding index for labels ***
# This value MUST be ignored by your loss function.
# For nn.CrossEntropyLoss, use its 'ignore_index' parameter (default -100 is common).
# Choose a value that doesn't conflict with valid label indices/values.
LABEL_PADDING_IDX = -1000000

def pad_collate_fn_time_step_level(batch):
    sequences = [item[0] for item in batch]
    # Labels are also sequences now
    labels = [item[1] for item in batch] # e.g., list of tensors of shape [Li] or [Li, C]
    lengths = torch.tensor([len(seq) for seq in sequences], dtype=torch.long) # Lengths based on input sequences

    # Pad features (input sequences)
    padded_sequences = pad_sequence(sequences, batch_first=True, padding_value=0.0)

    # *** Pad labels (label sequences) ***
    # Use the dedicated label padding index!
    padded_labels = pad_sequence(labels, batch_first=True, padding_value=LABEL_PADDING_IDX)

    return padded_sequences, lengths, padded_labels

# Usage:
# train_loader = DataLoader(..., collate_fn=pad_collate_fn_time_step_level)

In [6]:
def plot_rul_vs_time(unit_nr, model, dataloader):
    """
    Plot the RUL over time for a given unit number.
    """
    X, true_rul = dataloader.dataset[unit_nr - 1]
    
    model.eval()
    predicted_rul = model(X).detach().numpy().squeeze(0)

    fig = px.line(
        x=np.arange(1, len(true_rul) + 1),
        y=true_rul.reshape(-1),
        title=f'RUL Over Time: Unit {unit_nr}',
        labels={'x': 'Time', 'y': 'RUL'},
        markers=False
    )
    fig.add_scatter(
        x=np.arange(1, len(predicted_rul) + 1),
        y=predicted_rul.reshape(-1),
        mode='lines',
        name='Predicted RUL'
    )
    fig.update_layout(
        legend=dict(yanchor="bottom",
                    y=0.1,
                    xanchor="left",
                    x=0.1)
    )
    fig.show()
    

In [7]:
train_dataLoader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=pad_collate_fn_time_step_level)
test_dataLoader = DataLoader(test_dataset, batch_size=100, shuffle=False, collate_fn=pad_collate_fn_time_step_level) 

<h2>LSTM Without Learnable Initial States</h2>

In [ ]:
hparams = {
    'LR': LR,
    'BatchSize': BATCH_SIZE,
    'HiddenSizes': [32, 64],
    'LayerSizes': [1, 1],
    'DropoutRate': 0.3,
    'LossFn': 'MSELoss',
    'Optimizer': 'RMSprop'
}
model = RUL_Model_LearnableStates(input_size=train_dataLoader.dataset[0][0].shape[1], 
                                  lstm_hidden_sizes=hparams['HiddenSizes'], 
                                  lstm_layer_sizes=hparams['LayerSizes'], 
                                  lstm_dropout_rate=hparams['DropoutRate'], 
                                  output_dropout_rate=hparams['DropoutRate'], state_init=True)
model.to(device)
loss_fn = torch.nn.MSELoss(reduction='none')  # mean-squared error for regression
# optimizer = torch.optim.AdamW(model.parameters(), lr=LR)
optimizer = torch.optim.RMSprop(model.parameters(), lr=LR)

# Just to use the original visualisation module
y_test = pd.DataFrame({"RUL": [test_dataset[i][1][-1].item() for i in range(100)]})


In [9]:

writer = SummaryWriter(log_dir='runs/model_L-3264-N-88__learnStates_stateInit_lr0p001_bs20_dp0p3_RMSprop_MSELoss_mps')
writer.add_graph(model, train_dataset[0][0].to(device))

final_train_loss, best_val_loss, last_epoch = train_model(
    model=model,
    train_dataLoader=train_dataLoader, # Your training DataLoader
    test_dataLoader=test_dataLoader,   # Your validation DataLoader
    loss_fn=loss_fn,
    optimizer=optimizer,
    device=device,
    writer=writer, # Pass your writer if using TensorBoard
    num_epochs=100,
    patience=10, # Example patience
    best_model_path='best_model.pth', # Choose save path
    y_test=y_test, # Optional: for visualization
)

# writer.add_hparams(hparam_dict=hparams, metric_dict={'Train Loss': train_loss,
#                                                      'Validation Loss': val_loss,
#                                                      'Num Epochs Run': epochs})

Using device: mps
Epoch   1/100 | Training... 100 | Train Loss (avg/element): 9707.8503 | Val RMSE (last step): 83.7468
Validation RMSE improved (inf --> 83.7468). Saving model...
Epoch   2/100 | Training... 100 | Train Loss (avg/element): 9281.2118 | Val RMSE (last step): 81.5148
Validation RMSE improved (83.7468 --> 81.5148). Saving model...
Epoch   3/100 | Training... 100 | Train Loss (avg/element): 8879.7499 | Val RMSE (last step): 79.6002
Validation RMSE improved (81.5148 --> 79.6002). Saving model...
Epoch   4/100 | Training... 100 | Train Loss (avg/element): 8533.4807 | Val RMSE (last step): 77.7341
Validation RMSE improved (79.6002 --> 77.7341). Saving model...
Epoch   5/100 | Training... 100 | Train Loss (avg/element): 8181.1430 | Val RMSE (last step): 75.8016
Validation RMSE improved (77.7341 --> 75.8016). Saving model...
Epoch   6/100 | Training... 100 | Train Loss (avg/element): 7829.4278 | Val RMSE (last step): 73.8593
Validation RMSE improved (75.8016 --> 73.8593). Saving

In [35]:
rul_seqlen_err = list() # np.zeros((100, 3))
model.eval()
with torch.no_grad():
    for i, (x, y) in enumerate(test_dataLoader):
        y_pred = model(x)[:, -1, :].item()
        true_rul = y[:, -1, :].item()
        rul_err = true_rul - y_pred
        rul_seqlen_err.append([true_rul, x.shape[1], rul_err])

rul_seqlen_err = np.array(rul_seqlen_err).T
rul_seqlen_err[2] = np.abs(rul_seqlen_err[2])

px.scatter(
    x=rul_seqlen_err[0],
    y=rul_seqlen_err[1],
    color=rul_seqlen_err[2],
    title='True RUL vs Sequence Length',
    labels={'x': 'True RUL', 'y': 'Sequence Length', 'color': 'RUL Error'},
    color_continuous_scale=px.colors.sequential.Viridis
).show()
px.scatter(
    x=rul_seqlen_err[0],
    y=rul_seqlen_err[2],
    color=rul_seqlen_err[1],
    title='True RUL vs RUL Error',
    labels={'x': 'True RUL', 'y': 'RUL Error', 'color': 'Sequence Length'},
    color_continuous_scale=px.colors.sequential.Viridis
).show()
px.scatter(
    x=rul_seqlen_err[1],
    y=rul_seqlen_err[2],
    color=rul_seqlen_err[0],
    title='Sequence Length vs RUL Error',
    labels={'x': 'Sequence Length', 'y': 'RUL Error', 'color': 'True RUL'},
    color_continuous_scale=px.colors.sequential.Viridis
).show()

# px.line(x=np.arange(1, len(true_rul) + 1), y=((true_rul - predicted_rul)).reshape(-1), title='RUL Error Over Time', labels={'x': 'Time', 'y': 'RUL Error'}, markers=False).show()